In [1]:
import edward as ed
import numpy as np
import tensorflow as tf
from edward.models import Normal
import autograd.numpy as np
import autograd.numpy.random as npr
from neuralfingerprint import load_data
from autograd import grad
from keras.layers import Embedding, Dense


Using TensorFlow backend.


In [2]:
task_params = {'target_name' : 'measured log solubility in mols per litre',
               'data_file'   : 'delaney.csv'}
N_train = 800
N_val   = 20
N_test  = 20

model_params = dict(fp_length=50,    # Usually neural fps need far fewer dimensions than morgan.
                    fp_depth=4,      # The depth of the network equals the fingerprint radius.
                    conv_width=20,   # Only the neural fps need this parameter.
                    h1_size=100,     # Size of hidden layer of network on top of fps.
                    L2_reg=np.exp(-2))

print "Loading data..."
traindata, valdata, testdata = load_data(
    task_params['data_file'], (N_train, N_val, N_test),
    input_name='smiles', target_name=task_params['target_name'])
train_inputs, train_targets = traindata
val_inputs,   val_targets   = valdata
test_inputs,  test_targets  = testdata

print "Task params", task_params

Loading data...
Task params {'target_name': 'measured log solubility in mols per litre', 'data_file': 'delaney.csv'}


In [3]:
# X_train = train_inputs[0:10]
# y_train = train_targets[0:10]

In [4]:
# def gru_cell(hprev, xt):
#     #  output gate
#     import pdb; pdb.set_trace()
#     i_o = tf.sigmoid(tf.matmul(xt,Wfo) + tf.matmul(hprev,Wro) )
#     #  forget gate
#     i_f = tf.sigmoid(tf.matmul(xt,Wff) + tf.matmul(hprev,Wrf) )
#     #  intermediate
#     y = tf.tanh(tf.matmul(xt,Wfy) + tf.matmul( (i_f*hprev),Wry) )
#     # new state
#     return (1-i_o)*y + (i_o*hprev)

In [38]:
from edward.models import *
from edward.util import Progbar
H = 5
D = 2
V = 10
E = 2
batch_size = 1000
M = 10
nb_steps = 1
# X_train = np.arange(0,10000,1,dtype=np.int32)
# y_train = np.arange(0,10000,1,dtype=np.int32)
# X_train = X_train.reshape(10000,1)
# y_train = y_train.reshape(10000,1)
# X_train = np.ones([10000, nb_steps ], dtype=np.int32)
# y_train = np.ones([10000, 1 ], dtype=np.int32)
X_train = np.reshape(np.arange(0,10000),[10000, 1])
y_train = 2*X_train

In [40]:
# Wfo = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
# Wro = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

# Wff = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
# Wrf = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

# Wfy = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
# Wry = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

In [41]:
with tf.variable_scope('model', reuse=True):

    ##### params ######
    Wfo = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
    Wro = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

    Wff = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
    Wrf = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

    Wfy = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
    Wry = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

    qWfo = Normal(loc=tf.Variable(tf.random_normal([D, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H]))))
    qWro = Normal(loc=tf.Variable(tf.random_normal([H, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H]))))

    qWff = Normal(loc=tf.Variable(tf.random_normal([D, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H]))))
    qWrf = Normal(loc=tf.Variable(tf.random_normal([H, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H]))))

    qWfy = Normal(loc=tf.Variable(tf.random_normal([D, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H]))))
    qWry = Normal(loc=tf.Variable(tf.random_normal([H, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H]))))
    ####################
                  
    y_ph = tf.placeholder(tf.float32, [batch_size, 1], name='y_ph')

    x = tf.placeholder(tf.int32, [batch_size, nb_steps ], name='x')     

    def gru_cell(hprev, xt):
        #  output gate
        #import pdb; pdb.set_trace()
        i_o = tf.sigmoid(tf.matmul(xt,Wfo) + tf.matmul(hprev,Wro) )
        #  forget gate
        i_f = tf.sigmoid(tf.matmul(xt,Wff) + tf.matmul(hprev,Wrf) )
        #  intermediate
        y = tf.tanh(tf.matmul(xt,Wfy) + tf.matmul( (i_f*hprev),Wry) )
        # new state
        return (1-i_o)*y + (i_o*hprev)
      
    # embed -->
    x_ = Embedding(V, D, name='Embedding')(x)
    
    # initial state of RNN
    h = tf.zeros(shape=(batch_size, H)) # initial state

    for t in range(nb_steps-1):
        h = gru_cell(h, x_[:,t,:])

    ############# Varitational Inference #########################  
    W1 = Normal(loc=tf.zeros([D, 1]), scale=tf.ones([D, 1]))
    W2 = Normal(loc=tf.zeros([H, D]), scale=tf.ones([H, D]))
    
    qW1 = Normal(loc=tf.Variable(tf.random_normal([D, 1])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, 1]))))
    qW2 = Normal(loc=tf.Variable(tf.random_normal([H, D])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, D]))))
    
    def fhw(h_in):
        fhw = tf.matmul(tf.sigmoid(tf.matmul(h_in, W2)), W1)    
        return fhw
    #######################################################   
    #import pdb; pdb.set_trace()
    y = Normal(loc=fhw(h), scale=0.1 * tf.ones([batch_size,1]))


inference = ed.KLqp({W1: qW1, W2: qW2, 
                     Wfo: qWfo, Wro: qWro, 
                     Wff: qWff, Wrf: qWrf, 
                     Wfy: qWfy, Wry: qWry}, data={y: y_ph})


In [42]:
# W1 = Normal(loc=tf.zeros([D, D]), scale=tf.ones([D, D]))
# W2 = Normal(loc=tf.zeros([D, K]), scale=tf.ones([D, K]))

# fhw = tf.matmul(W2,tf.sigmoid(tf.matmul(W1, h)))

In [43]:
### Run -->
optimizer = tf.train.RMSPropOptimizer(0.01, epsilon=1.0)
inference.initialize(optimizer=optimizer,scale={y: len(X_train) / batch_size}) # always redefine inference before
sess = ed.get_session()
tf.global_variables_initializer().run()

n_epoch = 100
n_iter_per_epoch = 100

for epoch in range(n_epoch):
  avg_loss = 0.0

  pbar = Progbar(n_iter_per_epoch)
  for t in range(1, n_iter_per_epoch + 1):
    pbar.update(t)   
    batch = np.random.randint(0, len(X_train)-1, batch_size)
    info_dict = inference.update({x: X_train[batch], y_ph: y_train[batch]})
    avg_loss  += info_dict['loss']
    
  # Print a lower bound to the average marginal likelihood for an
  # image.
  avg_loss = avg_loss / n_iter_per_epoch
  avg_loss = avg_loss / batch_size
  print("log p(x) >= {:0.3f}".format(avg_loss))

100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66335891982.909
100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66749178063.094
100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66754221336.166
100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66849911278.141
100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66433454597.734
100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66877502584.259
100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66524493727.662
100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66745219059.548
100/100 [100%] ██████████████████████████████ Elapsed: 0s 0
log p(x) >= 66663316843.397
100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66720587338.220
100/100 [100%] ██████████████████████████████ Elapsed: 0s
log p(x) >= 66672685786.071
100/100 [100%] ██████████████████████████████ Elapse

In [47]:
## do test

# X_test = np.zeros([1000, nb_steps ])
X_test = np.reshape(np.arange(0,1000),[1000, 1])
test1 = sess.run({W2: qW2.mean()})


In [48]:
v = y.mean()
v2 = v.eval()
print(v2)

[[0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]
 [0.3561164]

In [49]:
dd = y.sample()
dd = dd.eval()
print(dd)

[[0.49479198]
 [0.6108664 ]
 [0.6709634 ]
 [0.62656116]
 [0.74866813]
 [0.66946906]
 [0.6818992 ]
 [0.6607934 ]
 [0.49239337]
 [0.70270836]
 [0.40000516]
 [0.7254624 ]
 [0.5639183 ]
 [0.6089495 ]
 [0.7605379 ]
 [0.69917923]
 [0.7489727 ]
 [0.7340197 ]
 [0.6658729 ]
 [0.40387598]
 [0.71776164]
 [0.6820465 ]
 [0.7200705 ]
 [0.687842  ]
 [0.73816025]
 [0.53740287]
 [0.44668776]
 [0.6274922 ]
 [0.6347563 ]
 [0.69960356]
 [0.5395798 ]
 [0.73232275]
 [0.57824653]
 [0.6062295 ]
 [0.70006293]
 [0.53442174]
 [0.59912086]
 [0.638942  ]
 [0.60402554]
 [0.76332694]
 [0.8012592 ]
 [0.70462817]
 [0.6624092 ]
 [0.8009823 ]
 [0.6746977 ]
 [0.5354035 ]
 [0.8069939 ]
 [0.63479096]
 [0.7515145 ]
 [0.7645267 ]
 [0.79546463]
 [0.69045264]
 [0.75849855]
 [0.8103806 ]
 [0.6400488 ]
 [0.5783826 ]
 [0.68838257]
 [0.63776046]
 [0.6621669 ]
 [0.56071144]
 [0.88130385]
 [0.6220976 ]
 [0.38741645]
 [0.60552347]
 [0.76006037]
 [0.59510535]
 [0.78796905]
 [0.7254865 ]
 [0.62638396]
 [0.6871292 ]
 [0.63222015]
 [0.76

In [41]:
import matplotlib.pyplot as plt
plt.hist(dd)

ImportError: No module named matplotlib.pyplot

In [ ]:
## do test
X_test = np.ones([100, nb_steps])
y_test = np.ones([100, nb_steps])
test1 = sess.run({W1: qW1.sample(),W2: qW2.sample()},{x: X_test})

y_post = ed.copy(y, {W1: qW1,W2: qW2})
y_out = sess.run(y_post, feed_dict={x:X_test})
np.mean(np.square(y_out-y_test))